# Spotify Afrobeats Recommendation System
By Afolabi Cardoso

## Data Gathering

This notebook features the data gathering process. Using the Spotify API and Spotipy library, I was able to create functions to collect the useful metadata and audio features from the tracks in the playlist. After collecting the features as a dataframe, I exported them as a csv

#### Import Libraries

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spotipy
from spotipy.oauth2 import SpotifyClsientCredentials

from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

import time

#### Authentication

To gain access to the Spotify API, I'll use the client_id and client_secret gotten from my developer account

In [5]:
c_id = 'My_ID'
c_secret = 'My_Secret'

In [6]:
credentials = SpotifyClientCredentials(client_id = c_id,
                                                      client_secret = c_secret)

#### Spotipy Object

Using the client credentials created above, I'll create a spotipy object sp

In [14]:
sp = spotipy.Spotify(client_credentials_manager = credentials)

#### Get track metadata and features

This function takes in a playlist URI and returns a dataframe containing metadata such as artist name, genre, album name, track_uri, danceability, energy, loudness, instrumentalness etc

In [15]:
def get_track_info(playlist):
    
    #split the playlist_uri is at the end of the playlists url. I'll use the .split method to extract it
    uri = playlist.split("/")[-1].split("?")[0]
    
    #from the spotipy library, use the playlist_tracks() method to extract each track from the playlist uri
    #It comes in a nested dictionary format
    
    results = sp.playlist_tracks(uri)
    tracks = results['items']
    
    #I need to use this while loop so that i'm not constrained to the 100 song limit by the spotify api 
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    
    
    #create an empty dictionary with the info we want to extract as columns
    info = {
    'track_uri':[],
    'track_name':[],
    'artist_name':[],
    #'artist_info':[],
    'artist_uri':[],
    #'artist_popularity':[],
    #'artist_genre':[],
    'album':[],
    'track_pop':[],
    'year_released':[]
    }
    
    features = {'danceability': [],
     'energy': [],
     'key': [],
     'loudness': [],
     'mode': [],
     'speechiness': [],
     'acousticness': [],
     'instrumentalness': [],
     'liveness': [],
     'valence': [],
     'tempo': [],
     'type': [],
     'id': [],
     'uri': [],
     'track_href': [],
     'analysis_url': [],
     'duration_ms': [],
     'time_signature': []
               }
    
    #using a for loop, get the the info for each song and put it into the empty dictionary
    for track in tracks:
        #URI
        info['track_uri'].append((track["track"]["uri"]).split(':')[2])

        #Track name
        info['track_name'].append(track["track"]["name"])

        #Main Artist
        info['artist_uri'].append((track["track"]["artists"][0]["uri"]).split(':')[2])
        #info['artist_info'].append(sp.artist(track["track"]["artists"][0]["uri"]))

        #Name, popularity, genre
        info['artist_name'].append(track["track"]["artists"][0]["name"])
        #info['artist_popularity'].append(sp.artist(track["track"]["artists"][0]["uri"])["popularity"])
        #info['artist_genre'].append(sp.artist(track["track"]["artists"][0]["uri"])["genres"])

        #Album
        info['album'].append(track["track"]["album"]["name"])

        #Popularity of the track, year released
        info['track_pop'].append(track["track"]["popularity"])
        info['year_released'].append(track["track"]['album']['release_date'])
        
        #Transform the info dictionary into a dataframe
        info_df = pd.DataFrame(info)
        
        #using the track uri get the audio features and assign it to the empty dictionary 
        track_uri = track["track"]["uri"].split(':')[2] 
        
        try:
            for key,value in (sp.audio_features(track_uri)[0]).items():
                features[key].append(value)
            
        except:
            print(f'failed on track {track["track"]["name"]}')
            continue
        time.sleep(3)
        
    #Transform the features dictionary into a dataframe
    features_df = pd.DataFrame(features)
    
        
    
    return info_df.join(features_df)
        

#### Add label to the playlist dataframe


This function calls the get_track_info function above and creates a column with the playlist genre or given username

In [33]:
def raw_data(user_playlist_url, genre):
    user_playlist_info = get_track_info(user_playlist_url)
    #add user genre
    user_playlist_info.loc[:,'genre'] = genre
    return user_playlist_info

#### Afrobeats playlist

In the spotify music app, I manually created an playlist that contains 1500+ afrobeats tracks. Using the functions above, I will fetch the tracks metadata and features

In [34]:
#assign the playlist link to the variable afrobeats_playlist_url
afrobeats_playlist_url= "https://open.spotify.com/playlist/5ZCzd0nCLqiIX1jwQWfazW"

In [35]:
#run the raw_data function to get the playlist metadata and assign in to the dataframe afrobeats_df
afrobeats_df = raw_data(afrobeats_playlist_url, 'afrobeats')

failed on track Lonely


In [36]:
#display the a sample of the dataframe
afrobeats_df.head()

,track_uri,track_name,artist_name,artist_uri,album,track_pop,year_released,danceability,energy,key,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genre
0,0mDRuZmReEm6DquPLJlcEm,Oleku (feat. Brymo),Ice Prince,1sSt1DqqqFLkPwfrqafVyn,Oleku (feat. Brymo),44,2010-09-21,0.544,0.730,1.0,...,0.751,162.112,audio_features,0mDRuZmReEm6DquPLJlcEm,spotify:track:0mDRuZmReEm6DquPLJlcEm,https://api.spotify.com/v1/tracks/0mDRuZmReEm6...,https://api.spotify.com/v1/audio-analysis/0mDR...,291364.0,5.0,afrobeats
1,1tvi8tv0eykhNcV1WtaIqO,Move Back,5five,37zb1JQnDV9dRLatrASEj1,Move Back,42,2011-04-08,0.793,0.734,11.0,...,0.793,125.032,audio_features,1tvi8tv0eykhNcV1WtaIqO,spotify:track:1tvi8tv0eykhNcV1WtaIqO,https://api.spotify.com/v1/tracks/1tvi8tv0eykh...,https://api.spotify.com/v1/audio-analysis/1tvi...,237107.0,4.0,afrobeats
2,5YgyvTZmdkIfFOnj8ri8mZ,Pakurumo,WizKid,3tVQdUvClmAT7URs9V3rsp,Superstar,34,2011-07-01,0.879,0.737,2.0,...,0.967,119.990,audio_features,5YgyvTZmdkIfFOnj8ri8mZ,spotify:track:5YgyvTZmdkIfFOnj8ri8mZ,https://api.spotify.com/v1/tracks/5YgyvTZmdkIf...,https://api.spotify.com/v1/audio-analysis/5Ygy...,212006.0,4.0,afrobeats
3,2cWdhA4ZDBvYuHRKKIPT3k,The Matter (feat. Wizkid),Maleek Berry,520qA5VGL9iI0SbmEnTVNg,The Matter (feat. Wizkid),46,2013-03-10,0.697,0.841,2.0,...,0.808,103.070,audio_features,2cWdhA4ZDBvYuHRKKIPT3k,spotify:track:2cWdhA4ZDBvYuHRKKIPT3k,https://api.spotify.com/v1/tracks/2cWdhA4ZDBvY...,https://api.spotify.com/v1/audio-analysis/2cWd...,198058.0,4.0,afrobeats
4,4voOsb8qpJ569JaMiGAyrl,Slow Down (feat. Wiz Kid),R2Bees,0LFsP7WPfu5inz9a1amcE4,Refuse to Be Broke: Da Revolution 2,49,2014,0.520,0.717,6.0,...,0.704,135.550,audio_features,4voOsb8qpJ569JaMiGAyrl,spotify:track:4voOsb8qpJ569JaMiGAyrl,https://api.spotify.com/v1/tracks/4voOsb8qpJ56...,https://api.spotify.com/v1/audio-analysis/4voO...,261547.0,5.0,afrobeats


In [37]:
#get the length of the dataframe
print(f'Total tracks = {len(afrobeats_df)}')

Total tracks = 1871


In [38]:
#check for null values
print(f'Total number of null values = {afrobeats_df.isna().sum().sum()}')

Total number of null values = 18


In [40]:
#drop null values
afrobeats_df.dropna(inplace=True)

In [41]:
#confirm null values were dropped
print(f'Total number of null values = {afrobeats_df.isna().sum().sum()}')

Total number of null values = 0


#### Export the afrobeats playlist as a csv

In [43]:
afrobeats_df.to_csv('../data/afrobeats.csv', index = False)

#### Jacks playlist

My classmate Jack volunteered his spotify playlist which contains mostly classical music. 

In [44]:
jacks_playlist_url = 'https://open.spotify.com/playlist/6UlskZAcTPzcGMnQaMnIVm?si=5cbb031d1fdd4064'

In [45]:
jacks_playlist_df = raw_data(jacks_playlist_url, 'jack')

In [46]:
jacks_playlist_df.head()

,track_uri,track_name,artist_name,artist_uri,album,track_pop,year_released,danceability,energy,key,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genre
0,1YzcrcgR3T2RwAZg5tSvYP,Die Walküre / Erster Aufzug: Orchestervorspiel,Richard Wagner,1C1x4MVkql8AiABuTw6DgE,Solti - Wagner - The Operas,14,2012-01-01,0.2650,0.2830,2,...,0.1640,113.033,audio_features,1YzcrcgR3T2RwAZg5tSvYP,spotify:track:1YzcrcgR3T2RwAZg5tSvYP,https://api.spotify.com/v1/tracks/1YzcrcgR3T2R...,https://api.spotify.com/v1/audio-analysis/1Yzc...,196000,3,jack
1,6JmduA0I9QYtD1RiHQgWjj,"Götterdämmerung, WWV 86D, Act III: Siegfrieds ...",Richard Wagner,1C1x4MVkql8AiABuTw6DgE,"Wagner: Götterdämmerung, WWV 86D",0,2018-11-09,0.1170,0.0495,5,...,0.0396,66.858,audio_features,6JmduA0I9QYtD1RiHQgWjj,spotify:track:6JmduA0I9QYtD1RiHQgWjj,https://api.spotify.com/v1/tracks/6JmduA0I9QYt...,https://api.spotify.com/v1/audio-analysis/6Jmd...,409787,3,jack
2,1U1i1HBJ5H8DY5J4fO8ySg,Tannhäuser: Overture,Richard Wagner,1C1x4MVkql8AiABuTw6DgE,Wagner: Orchestral Favourites,41,1994-01-01,0.0897,0.1600,4,...,0.0579,81.802,audio_features,1U1i1HBJ5H8DY5J4fO8ySg,spotify:track:1U1i1HBJ5H8DY5J4fO8ySg,https://api.spotify.com/v1/tracks/1U1i1HBJ5H8D...,https://api.spotify.com/v1/audio-analysis/1U1i...,853827,4,jack
3,5loYnrcJ1XTIbs0MXKntlr,"Götterdämmerung, WWV 86D, Prologue: Siegfrieds...",Richard Wagner,1C1x4MVkql8AiABuTw6DgE,"Wagner: Götterdämmerung, WWV 86D",0,2018-11-09,0.2270,0.1770,10,...,0.0622,128.729,audio_features,5loYnrcJ1XTIbs0MXKntlr,spotify:track:5loYnrcJ1XTIbs0MXKntlr,https://api.spotify.com/v1/tracks/5loYnrcJ1XTI...,https://api.spotify.com/v1/audio-analysis/5loY...,309053,3,jack
4,3ci6KfIfr3UiIRgJ3WVYhd,"Piano Concerto No. 1 in D Minor, Op. 15: I. Ma...",Johannes Brahms,5wTAi7QkpP6kp8a54lmTOq,Brahms: Piano Concerto No.1,30,2005-01-01,0.1870,0.1360,2,...,0.0395,87.459,audio_features,3ci6KfIfr3UiIRgJ3WVYhd,spotify:track:3ci6KfIfr3UiIRgJ3WVYhd,https://api.spotify.com/v1/tracks/3ci6KfIfr3Ui...,https://api.spotify.com/v1/audio-analysis/3ci6...,1407000,4,jack


In [47]:
#check for null values
jacks_playlist_df.isna().sum()

track_uri           0
track_name          0
artist_name         0
artist_uri          0
album               0
track_pop           0
year_released       0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
type                0
id                  0
uri                 0
track_href          0
analysis_url        0
duration_ms         0
time_signature      0
genre               0
dtype: int64

#### Export Jack's playlist as a csv

In [48]:
jacks_playlist_df.to_csv('../data/jack.csv', index = False)

#### Ankita playlist

Ankita, also a classmate, gave me her spotify url. Her playlist is a mixture of pop and Indian music

In [49]:
ankita_playlist_url = 'https://open.spotify.com/playlist/6qzbkhrmxdFj5TtaXR0sfI?si=ElY40mMjQ7apOsHTGlQI7A'

In [50]:
ankita_playlist_df = raw_data(ankita_playlist_url, 'ankita')

In [51]:
ankita_playlist_df.head()

,track_uri,track_name,artist_name,artist_uri,album,track_pop,year_released,danceability,energy,key,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genre
0,068HSvCf5MbQfhV4qqaelg,"Haan Main Galat (From ""Love Aaj Kal"")",Pritam,1wRPtKGflJrBx9BmLsSwlU,"Haan Main Galat (From ""Love Aaj Kal"")",59,2020-01-29,0.685,0.736,0,...,0.893,117.979,audio_features,068HSvCf5MbQfhV4qqaelg,spotify:track:068HSvCf5MbQfhV4qqaelg,https://api.spotify.com/v1/tracks/068HSvCf5MbQ...,https://api.spotify.com/v1/audio-analysis/068H...,218644,4,ankita
1,2aHz87L2Z4a0ZEQ7vMlH8z,Badal Pe Paon Hai,Salim–Sulaiman,6ohaQzKaXrobAL8paLSaxq,Chak De India,53,2007-08-01,0.649,0.788,9,...,0.869,104.999,audio_features,2aHz87L2Z4a0ZEQ7vMlH8z,spotify:track:2aHz87L2Z4a0ZEQ7vMlH8z,https://api.spotify.com/v1/tracks/2aHz87L2Z4a0...,https://api.spotify.com/v1/audio-analysis/2aHz...,243931,4,ankita
2,0mACGvgvCcwjfJCPFLeabh,Ziddi Dil,Vishal Dadlani,6CXEwIaXYfVJ84biCxqc9k,Mary Kom,54,2018-05-27,0.666,0.833,11,...,0.541,99.007,audio_features,0mACGvgvCcwjfJCPFLeabh,spotify:track:0mACGvgvCcwjfJCPFLeabh,https://api.spotify.com/v1/tracks/0mACGvgvCcwj...,https://api.spotify.com/v1/audio-analysis/0mAC...,286093,4,ankita
3,0dT6g7oqjEqphjXMk8MKHz,Dangal,Pritam,1wRPtKGflJrBx9BmLsSwlU,Dangal (Original Motion Picture Soundtrack),56,2016-12-14,0.618,0.867,6,...,0.665,178.017,audio_features,0dT6g7oqjEqphjXMk8MKHz,spotify:track:0dT6g7oqjEqphjXMk8MKHz,https://api.spotify.com/v1/tracks/0dT6g7oqjEqp...,https://api.spotify.com/v1/audio-analysis/0dT6...,299326,4,ankita
4,77UjLW8j5UAGAGVGhR5oUK,Pray For Me (with Kendrick Lamar),The Weeknd,1Xyo4u8uXC1ZmMpatF05PJ,Black Panther The Album Music From And Inspire...,71,2018-02-09,0.735,0.677,2,...,0.188,100.584,audio_features,77UjLW8j5UAGAGVGhR5oUK,spotify:track:77UjLW8j5UAGAGVGhR5oUK,https://api.spotify.com/v1/tracks/77UjLW8j5UAG...,https://api.spotify.com/v1/audio-analysis/77Uj...,211440,4,ankita


In [53]:
ankita_playlist_df.isnull().sum()

track_uri           0
track_name          0
artist_name         0
artist_uri          0
album               0
track_pop           0
year_released       0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
type                0
id                  0
uri                 0
track_href          0
analysis_url        0
duration_ms         0
time_signature      0
genre               0
dtype: int64

#### Export Ankita's playlist as a csv

In [54]:
ankita_playlist_df.to_csv('../data/ankita.csv', index = False)

#### Playlist featuring top songs from Fela

Fela pioneered the Afrobeats sound. He called his sound Afrobeat (without the s). I will compare the difference in sounds in the EDA section

In [55]:
fela_playlist_url = 'https://open.spotify.com/playlist/1bSsiqaBobgbnZTTVEo4Qh?si=3093a84902434561'

In [56]:
fela_playlist_df = raw_data(fela_playlist_url, 'fela')

In [58]:
fela_playlist_df.head()

,track_uri,track_name,artist_name,artist_uri,album,track_pop,year_released,danceability,energy,key,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genre
0,11GDQVqIEKAB4QKOcIVOvG,Zombie,Fela Kuti,5CG9X521RDFWCuAhlo6QoR,Zombie,45,1977-01-01,0.712,0.942,2,...,0.585,131.812,audio_features,11GDQVqIEKAB4QKOcIVOvG,spotify:track:11GDQVqIEKAB4QKOcIVOvG,https://api.spotify.com/v1/tracks/11GDQVqIEKAB...,https://api.spotify.com/v1/audio-analysis/11GD...,745653,4,fela
1,6sNNtFKdCz0bnjx7IEXyl2,Expensive Shit,Fela Kuti,5CG9X521RDFWCuAhlo6QoR,Expensive Shit,38,1975-01-01,0.614,0.876,7,...,0.682,122.635,audio_features,6sNNtFKdCz0bnjx7IEXyl2,spotify:track:6sNNtFKdCz0bnjx7IEXyl2,https://api.spotify.com/v1/tracks/6sNNtFKdCz0b...,https://api.spotify.com/v1/audio-analysis/6sNN...,793200,4,fela
2,7nM9ZnOD6nL04E5dOjjZiw,Let's Start (feat. Ginger Baker) - Live,Fela Kuti,5CG9X521RDFWCuAhlo6QoR,Fela With Ginger Baker Live!,37,1971-01-01,0.631,0.843,7,...,0.743,102.327,audio_features,7nM9ZnOD6nL04E5dOjjZiw,spotify:track:7nM9ZnOD6nL04E5dOjjZiw,https://api.spotify.com/v1/tracks/7nM9ZnOD6nL0...,https://api.spotify.com/v1/audio-analysis/7nM9...,465280,4,fela
3,1HC6o3lTQvNmtH3ejYG4hs,Gentleman,Fela Kuti,5CG9X521RDFWCuAhlo6QoR,Gentleman,40,1973-01-01,0.601,0.819,10,...,0.767,86.621,audio_features,1HC6o3lTQvNmtH3ejYG4hs,spotify:track:1HC6o3lTQvNmtH3ejYG4hs,https://api.spotify.com/v1/tracks/1HC6o3lTQvNm...,https://api.spotify.com/v1/audio-analysis/1HC6...,881240,4,fela
4,4fSGItb7Y1uOGfSoZDadhn,Trouble Sleep Yanga Wake Am,Fela Kuti,5CG9X521RDFWCuAhlo6QoR,Roforofo Fight,35,1972-01-01,0.601,0.560,3,...,0.599,142.040,audio_features,4fSGItb7Y1uOGfSoZDadhn,spotify:track:4fSGItb7Y1uOGfSoZDadhn,https://api.spotify.com/v1/tracks/4fSGItb7Y1uO...,https://api.spotify.com/v1/audio-analysis/4fSG...,726027,4,fela


#### Exporta fela's playlist as a csv

In [59]:
fela_playlist_df.to_csv('../data/fela.csv', index = False)

#### Heavy metal playlist

In [60]:
heavy_metal_playlist_url = 'https://open.spotify.com/playlist/37i9dQZF1DX9qNs32fujYe?si=8686a89d3fb7415b'

In [61]:
heavy_metal_playlist_df = raw_data(heavy_metal_playlist_url, 'heavy_metal')

In [62]:
heavy_metal_playlist_df.head(2)

,track_uri,track_name,artist_name,artist_uri,album,track_pop,year_released,danceability,energy,key,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genre
0,2MuWTIM3b0YEAskbeeFE1i,Master Of Puppets,Metallica,2ye2Wgw4gimLv2eAKyk1NB,Master Of Puppets (Remastered),79,1986-03-03,0.543,0.836,4,...,0.56,105.173,audio_features,2MuWTIM3b0YEAskbeeFE1i,spotify:track:2MuWTIM3b0YEAskbeeFE1i,https://api.spotify.com/v1/tracks/2MuWTIM3b0YE...,https://api.spotify.com/v1/audio-analysis/2MuW...,515387,4,heavy_metal
1,1AtE4xrxLR7nSPWSk1VGqM,Sunset over the Empire,Arch Enemy,0DCw6lHkzh9t7f8Hb4Z0Sx,Sunset over the Empire,60,2022-05-20,0.390,0.942,0,...,0.17,90.216,audio_features,1AtE4xrxLR7nSPWSk1VGqM,spotify:track:1AtE4xrxLR7nSPWSk1VGqM,https://api.spotify.com/v1/tracks/1AtE4xrxLR7n...,https://api.spotify.com/v1/audio-analysis/1AtE...,243080,4,heavy_metal


#### Export heavy metal playlist as a csv

In [63]:
heavy_metal_playlist_df.to_csv('../data/heavymetal.csv', index = False)

#### Rock playlist

In [64]:
rock_playlist_url = 'https://open.spotify.com/playlist/37i9dQZF1DXcF6B6QPhFDv?si=53be43146b8f4585'

In [65]:
rock_playlist_df = raw_data(rock_playlist_url, 'rock')

In [66]:
rock_playlist_df.head(2)

,track_uri,track_name,artist_name,artist_uri,album,track_pop,year_released,danceability,energy,key,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genre
0,7sA2SKTo1QbTSSYn5YvJC4,Sharks,Imagine Dragons,53XhwfbYqKCa1cC15pYq2q,Mercury - Acts 1 & 2,63,2022-07-01,0.927,0.642,10,...,0.901,123.969,audio_features,7sA2SKTo1QbTSSYn5YvJC4,spotify:track:7sA2SKTo1QbTSSYn5YvJC4,https://api.spotify.com/v1/tracks/7sA2SKTo1QbT...,https://api.spotify.com/v1/audio-analysis/7sA2...,190883,4,rock
1,63WuQOXohQIxOxIFA2K7bR,SUPERMODEL,Måneskin,0lAWpj5szCSwM4rUMHYmrr,SUPERMODEL,87,2022-05-13,0.636,0.876,7,...,0.799,120.983,audio_features,63WuQOXohQIxOxIFA2K7bR,spotify:track:63WuQOXohQIxOxIFA2K7bR,https://api.spotify.com/v1/tracks/63WuQOXohQIx...,https://api.spotify.com/v1/audio-analysis/63Wu...,148064,4,rock


#### Export rock playlist as a csv

In [67]:
rock_playlist_df.to_csv('../data/rock.csv', index = False)

#### Jazz playlist

In [68]:
jazz_playlist_url = 'https://open.spotify.com/playlist/37i9dQZF1DXdwTUxmGKrdN?si=a054d3423fca4687'

In [69]:
jazz_playlist_df = raw_data(jazz_playlist_url, 'jazz')

In [70]:
jazz_playlist_df.head(2)

,track_uri,track_name,artist_name,artist_uri,album,track_pop,year_released,danceability,energy,key,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genre
0,3y9l6PAbOOGYBH1AY4XaTO,Perfect Sunday,Carol Albert,3o1J5y4sY6jg1sOvF69qnm,Stronger Now,51,2020-01-31,0.621,0.650,3,...,0.433,168.069,audio_features,3y9l6PAbOOGYBH1AY4XaTO,spotify:track:3y9l6PAbOOGYBH1AY4XaTO,https://api.spotify.com/v1/tracks/3y9l6PAbOOGY...,https://api.spotify.com/v1/audio-analysis/3y9l...,243367,4,jazz
1,1xLU1zCbjo9HAoutmWsHlK,Exhale,Cindy Bradley,5qiEZz19psMDf9UPQUao4J,The Little Things,41,2019-05-31,0.638,0.489,6,...,0.476,82.010,audio_features,1xLU1zCbjo9HAoutmWsHlK,spotify:track:1xLU1zCbjo9HAoutmWsHlK,https://api.spotify.com/v1/tracks/1xLU1zCbjo9H...,https://api.spotify.com/v1/audio-analysis/1xLU...,314575,4,jazz


#### Export jazz playlist as a csv

In [71]:
jazz_playlist_df.to_csv('../data/jazz.csv', index = False)

#### Country playlist

In [72]:
country_playlist_url = 'https://open.spotify.com/playlist/37i9dQZF1DWXdiK4WAVRUW?si=56683bfa3aca4788'

In [73]:
country_playlist_df = raw_data(country_playlist_url, 'country')

In [74]:
country_playlist_df.head(2)

,track_uri,track_name,artist_name,artist_uri,album,track_pop,year_released,danceability,energy,key,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genre
0,3fqwjXwUGN6vbzIwvyFMhx,Tennessee Whiskey,Chris Stapleton,4YLtscXsxbVgi031ovDDdh,Traveller,81,2015-05-04,0.392,0.37,9,...,0.512,48.718,audio_features,3fqwjXwUGN6vbzIwvyFMhx,spotify:track:3fqwjXwUGN6vbzIwvyFMhx,https://api.spotify.com/v1/tracks/3fqwjXwUGN6v...,https://api.spotify.com/v1/audio-analysis/3fqw...,293293,4,country
1,6foY66mWZN0pSRjZ408c00,Whiskey Glasses,Morgan Wallen,4oUHIQIBe0LHzYfvXNW4QM,If I Know Me,82,2018-04-27,0.614,0.68,6,...,0.707,149.959,audio_features,6foY66mWZN0pSRjZ408c00,spotify:track:6foY66mWZN0pSRjZ408c00,https://api.spotify.com/v1/tracks/6foY66mWZN0p...,https://api.spotify.com/v1/audio-analysis/6foY...,234347,4,country


#### Export country playlist as a csv

In [75]:
country_playlist_df.to_csv('../data/country.csv', index = False)

#### Hip Hop playlist

In [76]:
hiphop_playlist_url = 'https://open.spotify.com/playlist/37i9dQZF1DX97h7ftpNSYT?si=5b73c05e83c9410f'

In [77]:
hiphop_playlist_df = raw_data(hiphop_playlist_url, 'hiphop')

In [78]:
hiphop_playlist_df.head(2)

,track_uri,track_name,artist_name,artist_uri,album,track_pop,year_released,danceability,energy,key,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genre
0,0HCnj7pfrvshnocOJUFeit,Top Off,Gunna,2hlmm7s2ICUX0LVIhVFlZQ,Drip Season 3 (Deluxe),71,2018-02-02,0.906,0.395,1,...,0.376,146.056,audio_features,0HCnj7pfrvshnocOJUFeit,spotify:track:0HCnj7pfrvshnocOJUFeit,https://api.spotify.com/v1/tracks/0HCnj7pfrvsh...,https://api.spotify.com/v1/audio-analysis/0HCn...,197420,4,hiphop
1,68Dni7IE4VyPkTOH9mRWHr,No Role Modelz,J. Cole,6l3HvQ5sa6mXTsMTB19rO5,2014 Forest Hills Drive,88,2014-12-09,0.690,0.521,10,...,0.494,100.028,audio_features,68Dni7IE4VyPkTOH9mRWHr,spotify:track:68Dni7IE4VyPkTOH9mRWHr,https://api.spotify.com/v1/tracks/68Dni7IE4VyP...,https://api.spotify.com/v1/audio-analysis/68Dn...,292799,4,hiphop


#### Export Hip Hop playlist as a csv

In [79]:
hiphop_playlist_df.to_csv('../data/hiphop.csv', index = False)

#### RnB playlist

In [80]:
rnb_playlist_url = 'https://open.spotify.com/playlist/37i9dQZF1DWXbttAJcbphz?si=aa6c61a73e1a4c24'

In [81]:
rnb_playlist_df = raw_data(rnb_playlist_url, 'rnb')

In [82]:
rnb_playlist_df.head(2)

,track_uri,track_name,artist_name,artist_uri,album,track_pop,year_released,danceability,energy,key,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genre
0,0JXXNGljqupsJaZsgSbMZV,Sure Thing,Miguel,360IAlyVv4PCEVjgyMZrxK,All I Want Is You,79,2010-11-26,0.684,0.607,11,...,0.498,81.001,audio_features,0JXXNGljqupsJaZsgSbMZV,spotify:track:0JXXNGljqupsJaZsgSbMZV,https://api.spotify.com/v1/tracks/0JXXNGljqups...,https://api.spotify.com/v1/audio-analysis/0JXX...,195373,4,rnb
1,2xRYnHQdLD9ketvC01UYYT,It Won't Stop (feat. Chris Brown) - EP Version,Sevyn Streeter,6If57j6e3TXXk0HiLcIZca,"Call Me Crazy, But...",71,2013-12-03,0.657,0.574,1,...,0.211,119.992,audio_features,2xRYnHQdLD9ketvC01UYYT,spotify:track:2xRYnHQdLD9ketvC01UYYT,https://api.spotify.com/v1/tracks/2xRYnHQdLD9k...,https://api.spotify.com/v1/audio-analysis/2xRY...,281080,4,rnb


#### Export rnb playlist as a csv

In [83]:
rnb_playlist_df.to_csv('../data/rnb.csv', index = False)

The next notebook features me performing exploratory data analysis on the csv exported. 